In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')

In [ ]:
test = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# Droing target column for the training set
X_train = train.drop(labels = ['label'], axis = 1)

# The target column
y_train = train['label']

In [ ]:
y_train.value_counts()

In [ ]:
sns.countplot(y_train);

In [ ]:
X_train.isnull().sum().sum()
# great no null values

In [ ]:
y_train.isnull().sum().sum()

In [ ]:
# scaling the data 
X_train = X_train / 255.00
test = test / 255.0

- Each image of the numbers consist of 255 pixels so we deivde each one by 255 for Nueral Networks Later

In [ ]:
# Reshaping the data to 3 dimensions (height, width, canal)
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

- We reshaped the data to 3 dimensions (28px x 28px) instead of padnas DataFrame 1d vector 784 values
- this will make the computations much faster
 

In [ ]:
from keras.utils.np_utils import to_categorical

# Encoding Labels
y_train = to_categorical(y_train, num_classes = 10)

-  converted the label column from 3 as a number to [0,0,0,1,0,0,0,0,0,0]

In [ ]:
y_train[0]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Spliting the training data to train and test 
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1)

# Convolutional Model

- How CNN work
- Each feature is like a mini-image—a small two-dimensional array of values. Features match common aspects of the images. In the case of X images, features consisting of diagonal lines and a crossing capture all the important characteristics of most X’s. These features will probably match up to the arms and center of any image of an X ,After doing this for every feature pixel in every convolutional layer and every weight in every fully connected layer, the new weights give an answer that works slightly better for that image. This is then repeated with each subsequent image in the set of labeled images.

-This is great video by Codebasics he explained it well

https://www.youtube.com/watch?v=zfiSAzpy9NM&t=4s&ab_channel=codebasics

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Setting learning rate
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

epochs = 20
batch_size = 86

In [ ]:
history = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs,
                   validation_data = (X_test, y_test), verbose = 2)

visualizations of loss and accuracy 

In [ ]:
evaluation = model.evaluate(X_test,y_test)
print("validation_loss is {} , validation_acc is {}".format(evaluation[0],evaluation[1]))

In [ ]:
df_loss_acc = pd.DataFrame(history.history)
df_loss=df_loss_acc[['loss','val_loss']]
df_acc=df_loss_acc[['accuracy','val_accuracy']]

In [ ]:
df_loss.plot()

In [ ]:
df_acc.plot()

Preparing for confusion Matrix

In [ ]:
y_predicted = model.predict(X_test)
y_predicted[0]

In [ ]:
plt.matshow(X_test[0])

In [ ]:
np.argmax(y_predicted[0])

As you can see it's working well

In [ ]:
results = model.predict(test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("submissiom.csv",index=False)